In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import pandas as pd
import numpy as np
import config
from tqdm import tqdm
import warnings
from torch import nn, optim

from torchvision import models
import os
import torch.nn.functional as F
from dataset import DRDataset
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, cohen_kappa_score


In [3]:
def check_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    preds = []
    labels = []

    all_preds, all_labels = [], []
    num_correct = 0
    num_samples = 0
    
    with torch.no_grad():
        for data, targets, _ in loader:
            data = data.to(device=device)
            targets = targets.to(device=device)
            # print(data.shape)
            # print("tgt sh",targets.shape)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)

            preds.extend(predicted.cpu().numpy())
            labels.extend(targets.cpu().numpy())

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = correct / total
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

    model.train()

    return preds, labels, accuracy, precision, recall, f1





In [4]:
# Define paths and parameters
base_folder = "/home/shashank/All_code/Research/APTOS_Detection/output_results/ResNet50_RealSynth_new_pipeline"
subfolders = [os.path.join(base_folder, folder) for folder in os.listdir(base_folder)]
csv_path = "/home/shashank/All_code/Research/APTOS_Detection/output_results/20240616_202528_ResNet_newpipeline_192_REALSynth_log.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the CSV file
df = pd.read_csv(csv_path)

# Iterate over each subfolder
for subfolder in subfolders:
    if os.path.isdir(subfolder):
        # Extract batch size and learning rate from folder name
        folder_name = os.path.basename(subfolder)
        print(folder_name)
        _, _, date_time, batch_size, lr = folder_name.split('_')
        batch_size = int(batch_size)
        lr = float(lr)
        
        # Load the model checkpoint
        model_path = os.path.join(subfolder, "b3_.pth.tar")
        if os.path.exists(model_path):
            model = models.resnet50()
            model.fc = nn.Linear(model.fc.in_features, 5)
            checkpoint = torch.load(model_path)
            model.load_state_dict(checkpoint['state_dict'])
            model = model.to(device)
            
            
            test_ds = DRDataset(
            images_folder="/home/shashank/All_code/Research/APTOS_dataset/aptos2019_blindness_detection/Real_Synthetic/RS_val_split",
            path_to_csv="/home/shashank/All_code/Research/APTOS_dataset/aptos2019_blindness_detection/Real_Synthetic/RS_val_split.csv",
            transform=config.val_transforms,
            )
            
            test_loader = DataLoader(
            test_ds,
            batch_size=batch_size,
            num_workers=config.NUM_WORKERS,
            pin_memory=config.PIN_MEMORY,
            shuffle=True,
            )
            
            # Replace `your_dataset` with your actual dataset
            preds, labels, accuracy, precision, recall, f1 = check_accuracy(test_loader, model, device)
            print(f"Metrics for {folder_name} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

            # Update the CSV file
            df.loc[
                (df['batch_size'] == batch_size) & (df['Learning_rate'] == lr), 
                ['test_accuracy', 'test_precision', 'test_recall', 'test_f1']
            ] = [accuracy, precision, recall, f1]

# Save the updated CSV file
df.to_csv(csv_path, index=False)

resnet50RS_newpipeline20240616_210055_64_5e-05
Metrics for resnet50RS_newpipeline20240616_210055_64_5e-05 - Accuracy: 0.874043555032372, Precision: 0.8833715143775037, Recall: 0.874043555032372, F1: 0.875483239216745
resnet50RS_newpipeline20240616_211219_64_0.003
Metrics for resnet50RS_newpipeline20240616_211219_64_0.003 - Accuracy: 0.7628016480282519, Precision: 0.7996809566488253, Recall: 0.7628016480282519, F1: 0.7640809607790633
resnet50RS_newpipeline20240616_212236_64_0.0003
Metrics for resnet50RS_newpipeline20240616_212236_64_0.0003 - Accuracy: 0.8581518540317834, Precision: 0.8707925117378164, Recall: 0.8581518540317834, F1: 0.8587451144659307
resnet50RS_newpipeline20240617_002519_32_1e-05
Metrics for resnet50RS_newpipeline20240617_002519_32_1e-05 - Accuracy: 0.6851088875809299, Precision: 0.5700375331761335, Recall: 0.6851088875809299, F1: 0.6171269922172928
resnet50RS_newpipeline20240616_232541_32_5e-05
Metrics for resnet50RS_newpipeline20240616_232541_32_5e-05 - Accuracy: 0.8